In [73]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data = pd.read_csv('data_pddd.csv', delimiter='\t')
data = data[['title', 'artist', 'lyrics']].drop_duplicates()
data['lyrics'] = data['lyrics'].fillna('.')
remove_data = data[data['lyrics'] == '.'].index
data.drop(remove_data, inplace = True)
data = data.reset_index(drop = True)
data['ID'] = [i for i in range(len(data))]
data['words'] = data.lyrics.str.split(' ')
data.tail()

,title,artist,lyrics,ID,words
3954,잠시,방탄소년단,하루 때 행복 일상 속 사람 별일 곳 요즘 시간 덕 노래 노래 Yeah 노래 yea...,3954,"[하루, 때, 행복, 일상, 속, 사람, 별일, 곳, 요즘, 시간, 덕, 노래, 노..."
3955,잘할게,이승기,안 안 oh babe 술 전화 안 이제 안 안 안 oh babe 저녁 집 길 마음 ...,3955,"[안, 안, oh, babe, 술, 전화, 안, 이제, 안, 안, 안, oh, ba..."
3956,Stay,방탄소년단,꿈 방 주문 때 heart 순간 wherever yeah yeah 반복 복 속 손 ...,3956,"[꿈, 방, 주문, 때, heart, 순간, wherever, yeah, yeah,..."
3957,병,방탄소년단,커피 모금 불안 해소 endless rest 불편 행복 시간 하루 종일 잠 지금 p...,3957,"[커피, 모금, 불안, 해소, endless, rest, 불편, 행복, 시간, 하루..."
3958,그날에 나는 맘이 편했을까,이예준,일 때 모습 말 집 그날 맘 각오 숙제 사랑 필요 담배 얘기 그날 맘 각오 숙제 사...,3958,"[일, 때, 모습, 말, 집, 그날, 맘, 각오, 숙제, 사랑, 필요, 담배, 얘기..."


In [74]:
tfidf = TfidfVectorizer()
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['lyrics'])

In [75]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [76]:
indices = pd.Series(data.index, index=data[['title', 'artist']]).drop_duplicates()

In [77]:
def get_recommendations(title, artist, cosine_sim=cosine_sim):
    idx = indices[[(title, artist)]].values[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:4]
    movie_indices = [i[0] for i in sim_scores]
    return movie_indices
    #return data[['title', 'artist']].iloc[movie_indices]

In [78]:
result = []
for i in range(len(data)):
    result.append(get_recommendations(data.loc[i]['title'], data.loc[i]['artist']))
print(result)

[[1581, 2137, 2303], [872, 1398, 1765], [1752, 503, 2336], [2543, 110, 1235], [3886, 384, 1163], [3234, 45, 3270], [552, 1078, 138], [2932, 2405, 2828], [3761, 138, 952], [1212, 1855, 3792], [3252, 3129, 1600], [1111, 181, 59], [995, 2203, 3578], [353, 114, 328], [1741, 55, 2253], [298, 2212, 134], [1124, 2666, 3575], [268, 1603, 2325], [3720, 1910, 574], [789, 265, 1600], [769, 2497, 1394], [611, 2935, 1636], [596, 1622, 1798], [106, 2249, 1009], [88, 2272, 1133], [3457, 1703, 3211], [549, 38, 331], [2558, 570, 522], [1139, 1008, 464], [3608, 628, 1230], [969, 1119, 460], [64, 933, 3617], [2205, 1790, 1706], [3055, 2870, 3526], [931, 1478, 48], [2184, 2954, 1200], [2895, 839, 696], [3956, 2838, 1865], [2895, 321, 244], [3199, 2068, 110], [479, 235, 2037], [615, 464, 2578], [2408, 410, 2262], [739, 77, 464], [233, 1925, 3218], [615, 1270, 3234], [356, 1309, 1156], [87, 471, 803], [931, 464, 1600], [2500, 873, 3663], [1383, 3124, 101], [1139, 1008, 450], [2272, 1692, 615], [175, 2466, 2

In [79]:
data['sim'] = result

In [86]:
data.head()

,ID,title,artist,sim,words
0,0,좋은 날,아이유,"[1581, 2137, 2303]","[하늘, 오늘, 바람, 완벽, 못, 얘기, 시작, 말, 눈물, 고개, 말, 말, 하..."
1,1,Oh Yeah,GD&TOP,"[872, 1398, 1765]","[oh, yeah, oh, oh, yeah, top, oh, yeah, oh, oh..."
2,2,High High,GD&TOP,"[1752, 503, 2336]","[선수, 입장, class, mr, club, Boy, Stance, 금술, eve..."
3,3,니까짓게,씨스타,"[2543, 110, 1235]","[Si, si, sistar, brave, sound, play, game, pla..."
4,4,샤이보이,시크릿,"[3886, 384, 1163]","[뚜, 루, 압, 밥, 밥, 뚜, 루, 압, 밥, 밥, 뚜, 루, 압, 밥, 밥, ..."


In [85]:
data = data[['ID', 'title', 'artist', 'sim', 'words']]
data

,ID,title,artist,sim,words
0,0,좋은 날,아이유,"[1581, 2137, 2303]","[하늘, 오늘, 바람, 완벽, 못, 얘기, 시작, 말, 눈물, 고개, 말, 말, 하..."
1,1,Oh Yeah,GD&TOP,"[872, 1398, 1765]","[oh, yeah, oh, oh, yeah, top, oh, yeah, oh, oh..."
2,2,High High,GD&TOP,"[1752, 503, 2336]","[선수, 입장, class, mr, club, Boy, Stance, 금술, eve..."
3,3,니까짓게,씨스타,"[2543, 110, 1235]","[Si, si, sistar, brave, sound, play, game, pla..."
4,4,샤이보이,시크릿,"[3886, 384, 1163]","[뚜, 루, 압, 밥, 밥, 뚜, 루, 압, 밥, 밥, 뚜, 루, 압, 밥, 밥, ..."
...,...,...,...,...,...
3954,3954,잠시,방탄소년단,"[3020, 2800, 3302]","[하루, 때, 행복, 일상, 속, 사람, 별일, 곳, 요즘, 시간, 덕, 노래, 노..."
3955,3955,잘할게,이승기,"[1368, 3090, 1108]","[안, 안, oh, babe, 술, 전화, 안, 이제, 안, 안, 안, oh, ba..."
3956,3956,Stay,방탄소년단,"[37, 2838, 1865]","[꿈, 방, 주문, 때, heart, 순간, wherever, yeah, yeah,..."
3957,3957,병,방탄소년단,"[3559, 2942, 3396]","[커피, 모금, 불안, 해소, endless, rest, 불편, 행복, 시간, 하루..."


In [88]:
data.to_csv('data_sim.csv', sep='|', index=False)